In [42]:
import pandas as pd
import numpy as np

In [43]:
training_data = pd.read_csv('cleaned_rossman_data.csv')
training_data.head()

,Unnamed: 0,Store,DayOfWeek,day,month,year,Sales,Customers,Open,Promo,...,storeType_a,storeType_b,storeType_c,storeType_d,Assortment_a,Assortment_b,Assortment_c,public_holiday,easter,christmas
0,0,1,5,31,7,2015,5263,0.099708,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,2,5,31,7,2015,6064,0.112468,1,1,...,1,0,0,0,1,0,0,0,0,0
2,2,3,5,31,7,2015,8314,0.148195,1,1,...,1,0,0,0,1,0,0,0,0,0
3,3,4,5,31,7,2015,13995,0.271600,1,1,...,0,0,1,0,0,0,1,0,0,0
4,4,5,5,31,7,2015,4822,0.100437,1,1,...,1,0,0,0,1,0,0,0,0,0


In [44]:
training_data.head()

,Unnamed: 0,Store,DayOfWeek,day,month,year,Sales,Customers,Open,Promo,...,storeType_a,storeType_b,storeType_c,storeType_d,Assortment_a,Assortment_b,Assortment_c,public_holiday,easter,christmas
0,0,1,5,31,7,2015,5263,0.099708,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,2,5,31,7,2015,6064,0.112468,1,1,...,1,0,0,0,1,0,0,0,0,0
2,2,3,5,31,7,2015,8314,0.148195,1,1,...,1,0,0,0,1,0,0,0,0,0
3,3,4,5,31,7,2015,13995,0.271600,1,1,...,0,0,1,0,0,0,1,0,0,0
4,4,5,5,31,7,2015,4822,0.100437,1,1,...,1,0,0,0,1,0,0,0,0,0


In [45]:
training_data.drop("Unnamed: 0", axis=1, inplace=True)

In [46]:
training_data.shape

(718637, 23)

In [47]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_friedman1
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

In [48]:
Y = training_data.Sales.to_frame()
X = training_data
X.drop('Sales',axis=1,inplace=True)
X = X.to_numpy()
Y = Y.to_numpy()

In [8]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=3) 
TimeSeriesSplit(max_train_size=None, n_splits=3)
counter=0
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    name='X_train_'+str(counter)  
    np.save(name, X_train)
    name='X_test_'+str(counter) 
    np.save(name, X_test)
    name='y_test_'+str(counter) 
    np.save(name, y_test)
    name='y_train'+str(counter)  
    np.save(name, y_train)
    counter+=1

TRAIN: [     0      1      2 ... 179657 179658 179659] TEST: [179660 179661 179662 ... 359316 359317 359318]
TRAIN: [     0      1      2 ... 359316 359317 359318] TEST: [359319 359320 359321 ... 538975 538976 538977]
TRAIN: [     0      1      2 ... 538975 538976 538977] TEST: [538978 538979 538980 ... 718634 718635 718636]


In [10]:
est = GradientBoostingRegressor(n_estimators=400, learning_rate=0.1, \
                                max_depth=2, random_state=0, loss='ls').fit(X_train, y_train)

print('This is the MSE: ' + str(mean_squared_error(y_test, est.predict(X_test))))

/Users/mackdelany/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


This is the MSE: 979663.8332600966


In [18]:
est

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [9]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

In [11]:
def run_GBT(n_estimators, learning_rate, max_depth, random_state, loss):
    
    test_err = []
    train_err = []
    
    ytrain = 'y_train{}.npy'
    xtrain = 'X_train_{}.npy'
    ytest = 'y_test_{}.npy'
    xtest = 'X_test_{}.npy'
    
    for i in range(3): 

        y_train = np.load(ytrain.format(i)).reshape(-1)
        name = 'X_train_'+ str(i) + '.npy'
        x_train = np.load(xtrain.format(i)).reshape(len(y_train),-1)
        
        model = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=learning_rate, \
                                max_depth=max_depth, random_state=random_state, loss=loss)
              
        model.fit(x_train, y_train)
        y_test = np.load(ytest.format(i)).reshape(-1)
        x_test = np.load(xtest.format(i)).reshape(len(y_test),-1)
        y_hat = model.predict(x_test)
        
        print(y_hat)
        idx = y_test != 0
        test_err.append(metric (y_test[idx],y_hat[idx]))
        y_hat = model.predict(x_train)
        idx = y_train != 0
        train_err.append(metric (y_train[idx],y_hat[idx]))
        
   
    return test_err,train_err

In [4]:
results = pd.DataFrame(columns = ['n_estimators','learning_rate','max_depth',''])

results

,n_estimators,learning_rate,max_depth


In [13]:
for n_estimators in [10, 300, 400]:
    for learning_rate in [0.75, 0.2, 0.3]:
        for max_depth in [2, 3, 4]:
            print('Running GBT for...')
            print('n_estimators: ' + str(n_estimators))
            print('learning_rate:' + str(learning_rate))
            print('max_depth:' + str (max_depth))
            test_err,train_err = run_GBT(n_estimators, learning_rate, max_depth, random_state=0, loss='ls')
            print('Test Error:')
            print(test_err)
            print('Training Error')
            print(train_err)


Running GBT for...
n_estimators: 10
learning_rate:0.75
max_depth:2
[ 4098.33042049 10590.73036587  4919.3533836  ... 10372.79600456
  4182.08392524  3961.63960982]
[8827.2130647  9751.00843879 4577.71906816 ... 3249.97689724 5372.48620994
 5645.14493884]
[6427.02752943 7298.6420734  6427.02752943 ... 6398.65221922 9046.0991074
 6398.65221922]
Test Error:
[17.247565076466156, 118.85820868958565, 18.222666862412304]
Training Error
[17.305973935341942, 17.58700788502257, 17.829561958820513]
Running GBT for...
n_estimators: 10
learning_rate:0.75
max_depth:3
[ 3964.6807144  10728.28905329  4590.68959428 ...  8832.80415266
  3571.72760249  3932.69892642]
[9397.76468895 8390.41711971 4585.62093219 ... 3621.39356369 5337.61161959
 5093.90274592]
[6333.61734054 7028.66414256 6333.61734054 ... 5179.24594418 9036.49412618
 6693.73559852]
Test Error:
[16.94772909318486, 16.840007290957555, 16.75480863546174]
Training Error
[16.749572929715903, 16.614644164934216, 18.182401588126726]
Running GBT fo

Test Error:
[10.761055276560779, 9.792565745247602, 11.417428776137623]
Training Error
[9.2811968256813, 9.325879870719064, 9.533972419016626]
Running GBT for...
n_estimators: 300
learning_rate:0.3
max_depth:4
[ 3886.33101147 10338.55755929  4128.31646378 ...  8245.00732101
  3786.11787217  3176.80569797]
[9562.60453468 8858.86494214 5306.49853896 ... 3923.4800675  5866.4565831
 5619.4498491 ]
[6450.16653143 6972.79809517 6561.95727886 ... 5908.08857431 9123.94908571
 7135.04272636]
Test Error:
[9.182031791575078, 8.056874744226965, 10.668884025915073]
Training Error
[7.121850327466313, 7.45872672450682, 7.677540532099164]
Running GBT for...
n_estimators: 400
learning_rate:0.75
max_depth:2
[ 3745.44700891 10454.36175179  3786.89459005 ...  9299.65277514
  3694.3542922   3546.93476921]
[8601.59056005 9396.74572117 5388.49958668 ... 3803.15728217 6115.09321639
 5674.63278768]
[6662.14868458 6736.70842134 6910.49625478 ... 6241.41207331 8395.54535153
 7564.97742066]
Test Error:
[12.015170

In [49]:
model = GradientBoostingRegressor(n_estimators=400, learning_rate=0.3, \
                                max_depth=4, random_state=0, loss='ls')

In [50]:
model.fit(X,Y)

/Users/mackdelany/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.3, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=400, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [51]:
model

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.3, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=400, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [52]:
y_pred_gradient_boosted = model.predict(X)

In [53]:
metric(y_pred_gradient_boosted, Y)

7.3808897943056175

In [54]:
filename = 'gradient_boosted_model.pkl'

In [55]:
import pickle

In [56]:
with open(filename, 'wb') as f:
    pickle.dump(model,f) 

In [21]:
filename = 'gradient_boosted_model.sav'
pickle.dump(model,open(filename, 'wb'))